# 📌 Preparação das Tabelas Temporárias (TempViews)

Nessa etapa iniciaremos a preparação dos dados para a Camada Gold (Data Warehouse).

- 🔶 Ler os arquivos no DBFS (para não ficar lendo toda hora do S3 e consequentemente aumentar o custo por leitura)
- 🔶 Criar nossas Views Temporarias
- 🔶 Criar nossas View Temporaria da Estatisticas Por Jogador


## 🔶 Leitura dos Arquivos no DBFS

### 🔹 Dataframes das tabelas

In [0]:
# Ler os dados do DBFS em vez do S3
df_todas_partidas = spark.read.parquet("dbfs:/mnt/silver_temp/todas_partidas")
df_classificacao = spark.read.parquet("dbfs:/mnt/silver_temp/classificacao")
df_estatisticas = spark.read.parquet("dbfs:/mnt/silver_temp/estatisticas")

print('✅ Dataframes criado com sucesso a partir do DBFS')

✅ Dataframes criado com sucesso a partir do DBFS


## 🔶 Criação das Views Temporárias

### 🔹 TempViews Principais

In [0]:
# Recriar as views temporárias
df_classificacao.createOrReplaceTempView("classificacao")
df_estatisticas.createOrReplaceTempView("estatisticas")
df_todas_partidas.createOrReplaceTempView("todas_partidas")


### 💠 TempView 'Estatisticas_PorJogador'

- Refere-se ao DataFrame 'estatisticas'
- Agrupamos (`GROUP BY`) pelo Jogador, Camisa e Clube, pois é interessante somente os números gerais de cada Clube ao fim da competição 

In [0]:
 %sql
-- Query final para juntar todas as informações
CREATE OR REPLACE TEMP VIEW Estatisticas_PorJogador AS
SELECT
    e.jogador,
    e.clube, 
    e.numero_camisa,
    ROUND(avg(e.Minutos_em_Campo), 0) AS minutos_em_campo,
    ROUND(sum(e.Gols), 0) AS gols, 
    ROUND(sum(e.Assitencias), 0) AS ass, 
    ROUND(SUM(e.penaltis_batidos), 0) AS penaltis_batidos,  
    ROUND(SUM(e.penaltis_tentados), 0) AS penaltis_tentados, 
    ROUND(sum(e.total_chutes), 0) AS total_chutes,
    ROUND(sum(e.Chutes_a_Gol), 0) AS chutes_gol,
    ROUND(sum(e.toques_na_bola), 0) AS toques_na_bola,
    ROUND(sum(e.desarmes), 0) AS desarmes,
    ROUND(sum(e.interceptacoes), 0) AS interceptacoes,
    ROUND(sum(e.bloqueios), 0) AS bloqueios,
    ROUND(SUM(e.Gols_Esperados), 2) AS gols_esperados,
    ROUND(SUM(e.Assistencias_Esperadas), 2) AS ass_esperadas,
    ROUND(sum(e.acoes_de_criacao), 0) AS acoes_de_criacao,
    ROUND(sum(e.acoes_de_gol), 0) AS acoes_de_gol,
    ROUND(sum(e.passes_completos), 0) AS passes_completos,
    ROUND(sum(e.Passes_Tentados), 0) AS passes_tentados,
    ROUND(sum(e.Passes_Progressivos), 0) AS passes_progressivos,
    ROUND(sum(e.Conducoes_de_Bola), 0) AS conducoes_de_bola,
    ROUND(sum(e.Conducoes_Progressivas), 0) AS conducoes_progressivas,
    ROUND(sum(e.Dribles_Tentados), 0) AS dribles_tentados,
    ROUND(sum(e.Dribles_Completos), 0) AS dribles_completos
FROM estatisticas AS e
GROUP BY e.jogador, e.clube, e.numero_camisa;

### 🔹 TempView 'Base' (CTE)
- 📍 Essa view será a nossa Classificação só que um pouco resumida
- 📍 Também adicionamos uma coluna `tier_clubes` para separar os clubes por Tier

In [0]:
%sql
-- Criando a CTE (Common Table Expression) Base para calcular os TIERs dos clubes
CREATE OR REPLACE TEMP VIEW Base AS
SELECT 
    c.posicao,        -- Posição final do clube no campeonato
    c.clube,          -- Nome do clube
    -- Definição do TIER com base na posição na tabela
    CASE
        WHEN c.posicao BETWEEN 1 AND 5 THEN 'TIER S'   -- Top 3 clubes
        WHEN c.posicao BETWEEN 6 AND 10 THEN 'TIER A'   -- Do 4º ao 8º colocado
        WHEN c.posicao BETWEEN 11 AND 15 THEN 'TIER B'  -- Do 9º ao 16º colocado
        ELSE 'TIER C'                                  -- Últimos colocados (zona de rebaixamento)
    END AS tier_clubes,
    c.gols_marcados,  -- Total de gols marcados na temporada
    c.gols_sofridos   -- Total de gols sofridos na temporada
FROM classificacao AS c;

# 📌 Normalização das Métricas do Projeto

## 🔶 Normalização das Métricas Defensivas
Transformaremos diferentes métricas defensivas em uma escala comum de **0 a 10**, permitindo comparações mais justas entre os clubes.

---

### 🔢 Como funcionará a normalização?

#### ⚖️ Normalização Tradicional

A Normalização Tradicional é feita utilizando a fórmula:

![Normalização](https://github.com/Cavalheiro93/mvp-brasileirao-data-engineering/raw/main/images/notebook-images/Normalizacao.png)

Essa fórmula ajusta os valores para ficarem dentro do intervalo **0 a 10**.


#### **⚠️ Exceção para gols sofridos**

Como **menos gols sofridos** é **melhor**, aplicamos a inversão:

![Normalização](https://github.com/Cavalheiro93/mvp-brasileirao-data-engineering/raw/main/images/notebook-images/Normalizacao-invertida.png)

Isso garante que um time que sofreu menos gols tenha um valor **maior** na escala.

---

### 🏆 Explicação das Métricas Normalizadas
| Métrica            | Interpretação | Peso | 
|--------------------|---------------|----- | 
| `norm_gols_sofridos` | Times que sofreram menos gols terão valores maiores. | 60% 
| `norm_desarmes` | Times que fizeram mais desarmes terão valores maiores. | 25% 
| `norm_interceptacoes` | Times que interceptaram mais passes terão valores maiores. | 10% 
| `norm_bloqueios` | Times que bloquearam mais chutes terão valores maiores. | 5% 

---

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Normalizacao_Defensiva_Jogadores AS
SELECT 
    e.jogador,
    e.clube,
    e.numero_camisa,
    e.minutos_em_campo,

    -- Normalização da métrica: desarmes
    -- Fórmula: (valor - mínimo) / (máximo - mínimo) * 10
    -- Objetivo: escalar a quantidade de desarmes para uma faixa de 0 a 10
    ROUND(((e.desarmes - MIN(e.desarmes) OVER()) / 
         (MAX(e.desarmes) OVER() - MIN(e.desarmes) OVER()) * 10), 2) AS norm_desarmes,

    -- Normalização da métrica: interceptações
    -- Fórmula: (valor - mínimo) / (máximo - mínimo) * 10
    -- Objetivo: escalar a quantidade de interceptações para uma faixa de 0 a 10
    ROUND(((e.interceptacoes - MIN(e.interceptacoes) OVER()) / 
         (MAX(e.interceptacoes) OVER() - MIN(e.interceptacoes) OVER()) * 10), 2) AS norm_interceptacoes,

    -- Normalização da métrica: bloqueios
    -- Fórmula: (valor - mínimo) / (máximo - mínimo) * 10
    -- Objetivo: escalar a quantidade de bloqueios para uma faixa de 0 a 10
    ROUND(((e.bloqueios - MIN(e.bloqueios) OVER()) / 
         (MAX(e.bloqueios) OVER() - MIN(e.bloqueios) OVER()) * 10), 2) AS norm_bloqueios,

    -- Normalização invertida da métrica: gols sofridos pelo clube do jogador
    -- Fórmula: 10 - ((valor - mínimo) / (máximo - mínimo) * 10)
    -- Objetivo: times que sofrem menos gols contribuem positivamente pro score defensivo dos seus jogadores
    ROUND(10 - ((b.gols_sofridos - MIN(b.gols_sofridos) OVER()) / 
         (MAX(b.gols_sofridos) OVER() - MIN(b.gols_sofridos) OVER()) * 10), 2) AS norm_gols_sofridos_clube

FROM Estatisticas_PorJogador e
LEFT JOIN Base b ON e.clube = b.clube;


In [0]:
%sql
-- Criando a VIEW com o cálculo do Score Defensivo por jogador
CREATE OR REPLACE TEMP VIEW Score_Defensivo_Jogadores AS
SELECT 
    n.jogador,
    n.clube,
    n.numero_camisa,
    n.minutos_em_campo,
    n.norm_gols_sofridos_clube,
    n.norm_desarmes,
    n.norm_interceptacoes,
    n.norm_bloqueios,

    -- Cálculo do SCORE DEFENSIVO por jogador (média ponderada das métricas normalizadas)
    ROUND((
        (n.norm_gols_sofridos_clube * 0.6) +        -- Peso 60% para desempenho defensivo do clube
        (n.norm_desarmes * 0.25) +                  -- Peso 25% para desarmes
        (n.norm_interceptacoes * 0.10) +            -- Peso 10% para interceptações
        (n.norm_bloqueios * 0.05)                   -- Peso 5% para bloqueios
    ), 2) AS score_defensivo

FROM Normalizacao_Defensiva_Jogadores n;


## 🔶 Normalização das Métricas Ofensivas

Transformaremos diferentes métricas ofensivas em uma escala comum de **0 a 10**, permitindo comparações mais justas entre os clubes.

---



### 🔢 Como funcionará a normalização?

#### ⚖️ Normalização Tradicional

Usaremos a fórmula tradicional para normalizar os valores entre 0 e 10:

![Normalização](https://github.com/Cavalheiro93/mvp-brasileirao-data-engineering/raw/main/images/notebook-images/Normalizacao.png)

Nesse caso, **quanto maior o valor original**, melhor o desempenho ofensivo. Portanto, **não será necessário inverter nenhuma métrica**.

---


### 🏆 Métricas Selecionadas para Ofensivo

| Métrica              | Interpretação                                     | Peso |
|----------------------|---------------------------------------------------|------|
| `gols_marcados`           | Quantidade de gols marcados                                      | 25%  |
| `assistencias`            | Capacidade de criação de jogadas com resultado                   | 10%  |
| `penaltis_batidos`        | Presença ofensiva na área adversária                             | 2,5% |
| `total_chutes`            | Volume de tentativas ofensivas                                   | 2,5% |
| `chutes_a_gol`            | Qualidade das finalizações                                       | 7,5% |
| `xG` ou `gols_esperados`  | Qualidade esperada das finalizações                              | 15%  |
| `xA` ou `Ass_esperados`   | Qualidade esperada das assistências                              | 2,5% |
| `acoes_de_criacao`        | Jogadas que geram oportunidades                                  | 7,5% |
| `acoes_de_gol`            | Jogadas que terminam em gols ou quase isso                       | 15%  |
| `eficiência`              | Quantos gols foram convertidos em relação a gols esperados       | 15%  |

---

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Normalizacao_Ofensiva_Jogadores AS
SELECT 
    e.jogador,
    e.clube,
    e.numero_camisa,
    e.minutos_em_campo,

    -- Normalizações
    ROUND(((e.gols - MIN(e.gols) OVER()) / 
         (MAX(e.gols) OVER() - MIN(e.gols) OVER()) * 10), 2) AS norm_gols_marcados,

    ROUND(((e.ass - MIN(e.ass) OVER()) / 
         (MAX(e.ass) OVER() - MIN(e.ass) OVER()) * 10), 2) AS norm_ass,

    ROUND(((e.penaltis_batidos - MIN(e.penaltis_batidos) OVER()) / 
         (MAX(e.penaltis_batidos) OVER() - MIN(e.penaltis_batidos) OVER()) * 10), 2) AS norm_penaltis_batidos,

    ROUND(((e.total_chutes - MIN(e.total_chutes) OVER()) / 
         (MAX(e.total_chutes) OVER() - MIN(e.total_chutes) OVER()) * 10), 2) AS norm_total_chutes,

    ROUND(((e.chutes_gol - MIN(e.chutes_gol) OVER()) / 
         (MAX(e.chutes_gol) OVER() - MIN(e.chutes_gol) OVER()) * 10), 2) AS norm_chutes_gol,

    ROUND(((e.gols_esperados - MIN(e.gols_esperados) OVER()) / 
         (MAX(e.gols_esperados) OVER() - MIN(e.gols_esperados) OVER()) * 10), 2) AS norm_gols_esperados,

    ROUND(((e.ass_esperadas - MIN(e.ass_esperadas) OVER()) / 
         (MAX(e.ass_esperadas) OVER() - MIN(e.ass_esperadas) OVER()) * 10), 2) AS norm_ass_esperadas,

    ROUND(((e.acoes_de_criacao - MIN(e.acoes_de_criacao) OVER()) / 
         (MAX(e.acoes_de_criacao) OVER() - MIN(e.acoes_de_criacao) OVER()) * 10), 2) AS norm_acoes_de_criacao,

    ROUND(((e.acoes_de_gol - MIN(e.acoes_de_gol) OVER()) / 
         (MAX(e.acoes_de_gol) OVER() - MIN(e.acoes_de_gol) OVER()) * 10), 2) AS norm_acoes_de_gol,

    -- Eficiência de finalização: gols / xG
    ROUND((e.gols / NULLIF(e.gols_esperados, 0)), 2) AS eficiencia,

    -- Normalização da eficiência
    ROUND((
        ((e.gols / NULLIF(e.gols_esperados, 0)) - 
         MIN((e.gols / NULLIF(e.gols_esperados, 0))) OVER()) /
         (MAX((e.gols / NULLIF(e.gols_esperados, 0))) OVER() - 
         MIN((e.gols / NULLIF(e.gols_esperados, 0))) OVER()) * 10
    ), 2) AS norm_eficiencia

FROM Estatisticas_PorJogador e;


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Score_Ofensivo_Jogadores AS
SELECT 
    n.jogador,
    n.clube,
    n.numero_camisa,
    n.minutos_em_campo,

    -- Colunas de normalização
    n.norm_gols_marcados,
    n.norm_ass,
    n.norm_penaltis_batidos,
    n.norm_total_chutes,
    n.norm_chutes_gol,
    n.norm_gols_esperados,
    n.norm_ass_esperadas,
    n.norm_acoes_de_criacao,
    n.norm_acoes_de_gol,
    n.norm_eficiencia,

    -- Score final ofensivo com pesos atualizados
    ROUND((
        (n.norm_gols_marcados * 0.25) +         
        (n.norm_ass * 0.10) +                  
        (n.norm_penaltis_batidos * 0.025) +     
        (n.norm_total_chutes * 0.025) +         
        (n.norm_chutes_gol * 0.075) +           
        (n.norm_gols_esperados * 0.15) +       
        (n.norm_ass_esperadas * 0.025) +        
        (n.norm_acoes_de_criacao * 0.075) +     
        (n.norm_acoes_de_gol * 0.15) +         
        (n.norm_eficiencia * 0.15)             
    ), 2) AS score_ofensivo

FROM Normalizacao_Ofensiva_Jogadores n;


## 🔶 Normalização das Métricas de Criação

Transformaremos diferentes métricas de **criação de jogadas** em uma escala comum de **0 a 10**, permitindo entender quais clubes têm maior capacidade de construir oportunidades — mesmo que não convertam em gol.

---

### 🔢 Como funcionará a normalização?

Usaremos a fórmula tradicional de normalização:

![Normalização](https://github.com/Cavalheiro93/mvp-brasileirao-data-engineering/raw/main/images/notebook-images/Normalizacao.png)

Nenhuma métrica aqui será invertida, pois **quanto mais, melhor**.

### 🏆 Métricas Selecionadas para Criação

| Métrica                  | Interpretação                               | Peso |
|--------------------------|---------------------------------------------|------|
| `assistencias`           | Resultado direto de criação bem-sucedida    | 30%    |
| `acoes_de_criacao`       | Volume de jogadas que geram oportunidades   | 30%    |
| `xA`                     | Qualidade esperada das assistências         | 25%    |
| `passes_completos`       | Participação ativa na construção            | 3,75%  |
| `passes_tentados`        | Volume total de tentativas ofensivas        | 3,75%  |
| `conducoes_de_bola`      | Participação individual na construção       | 3,75%  |
| `conducoes_progressivas` | Avanço real com a bola no pé                | 3,75%  |

In [0]:
%sql
-- Criando a VIEW com a normalização das métricas de Criação por jogador
CREATE OR REPLACE TEMP VIEW Normalizacao_Criacao_Jogadores AS
SELECT 
    e.jogador,
    e.clube,
    e.numero_camisa,
    e.minutos_em_campo,

    -- Assistências
    ROUND(((e.ass - MIN(e.ass) OVER()) / 
        (MAX(e.ass) OVER() - MIN(e.ass) OVER()) * 10), 2) AS norm_assistencias,

    -- Ações de Criação
    ROUND(((e.acoes_de_criacao - MIN(e.acoes_de_criacao) OVER()) /
        (MAX(e.acoes_de_criacao) OVER() - MIN(e.acoes_de_criacao) OVER()) * 10), 2) AS norm_acoes_de_criacao,

    -- Assistências Esperadas (xA)
    ROUND(((e.ass_esperadas - MIN(e.ass_esperadas) OVER()) /
        (MAX(e.ass_esperadas) OVER() - MIN(e.ass_esperadas) OVER()) * 10), 2) AS norm_ass_esperadas,

    -- Passes Completos
    ROUND(((e.passes_completos - MIN(e.passes_completos) OVER()) /
        (MAX(e.passes_completos) OVER() - MIN(e.passes_completos) OVER()) * 10), 2) AS norm_passes_completos,

    -- Passes Tentados
    ROUND(((e.passes_tentados - MIN(e.passes_tentados) OVER()) /
        (MAX(e.passes_tentados) OVER() - MIN(e.passes_tentados) OVER()) * 10), 2) AS norm_passes_tentados,

    -- Conduções de Bola
    ROUND(((e.conducoes_de_bola - MIN(e.conducoes_de_bola) OVER()) /
        (MAX(e.conducoes_de_bola) OVER() - MIN(e.conducoes_de_bola) OVER()) * 10), 2) AS norm_conducoes_de_bola,

    -- Conduções Progressivas
    ROUND(((e.conducoes_progressivas - MIN(e.conducoes_progressivas) OVER()) /
        (MAX(e.conducoes_progressivas) OVER() - MIN(e.conducoes_progressivas) OVER()) * 10), 2) AS norm_conducoes_progressivas

FROM Estatisticas_PorJogador e;


In [0]:
%sql
-- Criando a VIEW com o cálculo do Score de Criação por jogador
CREATE OR REPLACE TEMP VIEW Score_Criacao_Jogadores AS
SELECT 
    n.jogador,
    n.clube,
    n.numero_camisa,
    n.minutos_em_campo,

    -- Métricas normalizadas
    n.norm_assistencias,
    n.norm_acoes_de_criacao,
    n.norm_ass_esperadas,
    n.norm_passes_completos,
    n.norm_passes_tentados,
    n.norm_conducoes_de_bola,
    n.norm_conducoes_progressivas,

    -- Score final com pesos ajustados
    ROUND((
        (n.norm_assistencias * 0.30) +
        (n.norm_acoes_de_criacao * 0.30) +
        (n.norm_ass_esperadas * 0.25) +
        (n.norm_passes_completos * 0.0375) +
        (n.norm_passes_tentados * 0.0375) +
        (n.norm_conducoes_de_bola * 0.0375) +
        (n.norm_conducoes_progressivas * 0.0375)
    ), 2) AS score_criacao

FROM Normalizacao_Criacao_Jogadores n;


# 📌 Criação dos Data marts

### 🧱 Mart_Desempenho_Jogadores

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Mart_Desempenho_Jogadores AS
SELECT 
    d.jogador,
    d.clube,
    d.numero_camisa,
    d.minutos_em_campo,

    -- Métricas defensivas
    d.norm_gols_sofridos_clube,
    d.norm_desarmes,
    d.norm_interceptacoes,
    d.norm_bloqueios,

    -- Métricas ofensivas
    o.norm_gols_marcados,
    o.norm_ass,
    o.norm_penaltis_batidos,
    o.norm_total_chutes,
    o.norm_chutes_gol,
    o.norm_gols_esperados,
    o.norm_ass_esperadas,
    o.norm_acoes_de_criacao,
    o.norm_acoes_de_gol,
    o.norm_eficiencia,

    -- Métricas de criação
    c.norm_passes_completos,
    c.norm_passes_tentados,
    c.norm_conducoes_de_bola,
    c.norm_conducoes_progressivas,

    -- SCORES
    d.score_defensivo,
    o.score_ofensivo,
    c.score_criacao

FROM Score_Defensivo_Jogadores d
LEFT JOIN Score_Ofensivo_Jogadores o 
    ON d.jogador = o.jogador AND d.clube = o.clube AND d.numero_camisa = o.numero_camisa
LEFT JOIN Score_Criacao_Jogadores c 
    ON d.jogador = c.jogador AND d.clube = c.clube AND d.numero_camisa = c.numero_camisa;


# 📌 Salvando os Arquivs na Camada Gold 

### 🔹Geral - Salvando Datamart na camada Gold

In [0]:
df_gold = spark.sql("SELECT * FROM Mart_Desempenho_Jogadores")

df_gold.write.mode("overwrite").parquet("s3a://mvp-brasileirao-2024/gold/Mart_Desempenho_Jogadores")